In [1]:
#READING CSV
import pandas as pd
from copy import deepcopy
from google.colab import drive 
drive.mount('/content/gdrive')

almacenes = pd.read_csv("/content/gdrive/MyDrive/Trabajos 7mo Ciclo/Complejidad/almacenes.csv")
alm = almacenes[['Coord X', 'Coord Y']]
nodeTypeALM = [1]*100
alm['NodeType'] = nodeTypeALM

puntos_entrega = pd.read_csv("/content/gdrive/MyDrive/Trabajos 7mo Ciclo/Complejidad/puntos_entrega.csv")
pe = puntos_entrega[['Coord X', 'Coord Y']]
nodeTypePE = [2]*5000
pe['NodeType'] = nodeTypePE

Mounted at /content/gdrive


In [2]:
#Join both lists
newDF = pd.concat([alm, pe])

#List to use (nodeType) and list to compare (noType)
nodeTypeList = newDF[['Coord X', 'Coord Y', 'NodeType']].values.tolist()
noTypeList = newDF[['Coord X', 'Coord Y']].values.tolist()

In [3]:
#Sorting List by Y then by X
def orderList(list_):
    newL = sorted(list_, key=lambda y: y[1])
    newL.sort(key=lambda x: x[0])
    return deepcopy(newL)

newL = orderList(noTypeList)
newTypeList = orderList(nodeTypeList)

In [4]:
# Storage/DeliveryPoint Nodes to append
def popStuff(popList, times):
    to_return = []
    popList.reverse()
    for j in range(times):
        to_return.append(popList.pop())
    popList.reverse()
    return to_return

#Search X Coords Nodes (it's sorted already)
def searchX(tempList, tempNodeTypeList, x):
    returnList = []
    popped = []
    max = 0

    if len(tempList)==0 and len(tempNodeTypeList)==0:
        return returnList, popped
    for i in tempList: #Getting comparison list
        if i[0] == x:
            max += 1
            returnList.append(i)
        else:
            break
    
    popStuff(tempList, max)
    popped = []
    popped = popStuff(tempNodeTypeList, max)
    return returnList, popped

In [5]:
# Create graph
def graph(nodesList, newL_, newTypeList_):
    xList = [i for i in range(1000)]
    yList = [i for i in range(1000)]
    nodesCont = 0

    tempNoTypeList = deepcopy(newL_)
    tempTypeList = deepcopy(newTypeList_)

    for x in xList:
        smallList = [] #Comparison list
        popped = [] #To List
        if len(tempNoTypeList)>0 and len(tempTypeList)>0:
            smallList, popped = searchX(tempNoTypeList, tempTypeList, x)
        for y in yList:
            ady1 = nodesCont - 1000 #left then goes clockwise
            ady2 = nodesCont - 1
            ady3 = nodesCont + 1000
            ady4 = nodesCont + 1
            # Validations - not corners
            if x == 0: ady1 = -1
            if x == 999: ady3 = -1
            if y == 0: ady2 = -1
            if y == 999: ady4 = -1

            ady0 = [ady1, ady2, ady3, ady4]

            if len(popped) > 0 and len(smallList) > 0 and [x, y] in smallList:
                t = popped.pop(0)
                nodesList.append([t[0], t[1], t[2], ady0])
            else:
                nodesList.append([x, y, 0, ady0])
            
            nodesCont += 1

In [6]:
# Creating the new graph
nodes = []
graph(nodes, newL, newTypeList)

In [7]:
# Making a copy of resulting graph
nodes_copy = deepcopy(nodes)

In [24]:
# BFS since all edges have the same weight (from a node to all others)
def bfs(graph, s, size):
    visited = [False]*size
    parent = [None]*size
    queue = [s]
    visited[s] = True

    while queue:
        u = queue.pop(0)
        node = graph[u]
        for v in node[3]:
            if v >= 0:
                if not visited[v]:
                    visited[v] = True
                    parent[v] = u
                    queue.append(v)

    return parent

def find_route(all_routes, alm_node, ent_node):
    start_node = all_routes[ent_node] #returns parent
    #ir desde ent_node hasta alm_node con un while, hasta que encuentre el nodo que necesita
    #Las rutas se obtienen recorriendo las rutas generales de manera inversa
    route = []
    route.append(ent_node)

    if start_node == alm_node: #Comprueba si son nodos adyacentes
        route.append(alm_node)
        route.reverse()
        return route

    while start_node != alm_node: #Recorre los padres desde el punto de entrega hasta el almacén (inversamente)
        route.append(start_node)
        start_node = all_routes[start_node]

    route.append(alm_node)
    route.reverse()
    return route #returns 1 route

def find_nodes(graph, node_type, max):
    nodes = []
    cont_ent = 0
    size = len(graph)
    for i in range(size):
        if graph[i][2] == node_type:
            nodes.append(i)
            cont_ent += 1
        if cont_ent == max:
            return nodes #returns all nodes of that type

In [26]:
def find_shorter_route(graph, alm_node, size, ent_nodes_list, shortest_routes):
    parent = bfs(graph, alm_node, size)

    for i in range(len(ent_nodes_list)): #ent_nodes
        route = []
        route.append(alm_node)
        route.append(ent_nodes_list[i])
        route.append(find_route(parent, alm_node, ent_nodes_list[i]))
        if len(shortest_routes) < len(ent_nodes_list): # Empty list
            shortest_routes.append(route)
        else:
            if len(route[2]) < len(shortest_routes[i][2]): #Check if it's a shorter route
                shortest_routes[i] = route # Replace if so

def shortest_routes(graph, size): #Only asking to every Storage node
    shortest_routes = []
    cont_alm = 0
    alm_nodes_list = find_nodes(graph, 1, 100) #alm = 1 / ent = 2
    ent_nodes_list = find_nodes(graph, 2, 5000) #alm = 1 / ent = 2

    for i in range(len(alm_nodes_list)):
        find_shorter_route(graph, alm_nodes_list[i], size, ent_nodes_list, shortest_routes)

    return shortest_routes #5k routes

In [27]:
routes = shortest_routes(nodes_copy, len(nodes_copy))

In [25]:
for i in range(5):
    print(routes[i])
print(len(routes))

[127153, 103, [127153, 126153, 125153, 124153, 123153, 122153, 121153, 120153, 119153, 118153, 117153, 116153, 115153, 114153, 113153, 112153, 111153, 110153, 109153, 108153, 107153, 106153, 105153, 104153, 103153, 102153, 101153, 100153, 99153, 98153, 97153, 96153, 95153, 94153, 93153, 92153, 91153, 90153, 89153, 88153, 87153, 86153, 85153, 84153, 83153, 82153, 81153, 80153, 79153, 78153, 77153, 76153, 75153, 74153, 73153, 72153, 71153, 70153, 69153, 68153, 67153, 66153, 65153, 64153, 63153, 62153, 61153, 60153, 59153, 58153, 57153, 56153, 55153, 54153, 53153, 52153, 51153, 50153, 49153, 48153, 47153, 46153, 45153, 44153, 43153, 42153, 41153, 40153, 39153, 38153, 37153, 36153, 35153, 34153, 33153, 32153, 31153, 30153, 29153, 28153, 27153, 26153, 25153, 24153, 23153, 22153, 21153, 20153, 19153, 18153, 17153, 16153, 15153, 14153, 13153, 12153, 11153, 10153, 9153, 8153, 7153, 6153, 5153, 4153, 3153, 2153, 1153, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 13

In [14]:
# def shortest_path(all_routes): #Shortest path to every delivery
#     #check every storage
#     shortest_routes = [] #Should have 5k routes - the shortest to every delivery
#     for storage in all_routes:
#         if shortest_routes == []:
#             for i in range(len(storage)):
#                 route = storage[i]
#                 shortest_routes.append(route) # Append first 5k routes (with no criteria)
#         else:
#             for i in range(len(storage)):
#                 route_to_compare = storage[i] # New 5k
#                 if len(shortest_routes[i][2]) > len(route_to_compare[2]): # Old 5k ~ new 5k routes
#                     shortest_routes[i] = route_to_compare
    
#     return shortest_routes

# shortest_routes = shortest_path(routes)

# print(len(shortest_routes))

# for i in range(30):
#     print(shortest_routes[i])